In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Flatten, LSTM, Dropout, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision, BinaryAccuracy
from tqdm import tqdm
from collections import defaultdict
from keras import backend as K
from scipy import stats
from itertools import chain
import scikit_posthocs as sp

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("final_13_5_2022.csv", index_col="Date", low_memory=False)

In [3]:
df

,spf_open,spf_high,spf_low,spf_close,spf_adj close,spf_volume,spf_sma_3,spf_wma_3,spf_trima_3,spf_ema_3,...,hous_starts,hous_permits,gdp_250,uk100,uk100_250,em,em_250,europe600,europe600_250,target
Date,,,,,,,,,,,,,,,,,,,,,
2000-09-18,1485.25,1489.75,1462.25,1467.50,1467.50,104794.0,1471.833333,1472.166667,1473.5000,1471.833333,...,1507.0,1570.0,0.027123,6410.200195,6410.200195,7.978623,0.000000,247.070007,0.000000,0
2000-09-19,1467.00,1482.75,1466.75,1478.50,1478.50,103371.0,1471.833333,1472.166667,1473.5000,1471.833333,...,1507.0,1570.0,0.027123,6403.500000,6403.500000,7.978623,0.000000,247.070007,0.000000,0
2000-09-20,1478.75,1480.50,1450.25,1469.50,1469.50,109667.0,1471.833333,1472.166667,1473.5000,1471.833333,...,1507.0,1570.0,0.027123,6279.899902,6279.899902,7.978623,0.000000,247.070007,0.000000,0
2000-09-21,1470.25,1474.00,1455.50,1469.50,1469.50,98528.0,1472.500000,1471.000000,1471.7500,1470.666667,...,1507.0,1570.0,0.027123,6199.200195,6199.200195,7.978623,0.000000,247.070007,0.000000,0
2000-09-22,1454.75,1471.00,1436.75,1468.50,1468.50,97416.0,1469.166667,1469.000000,1469.2500,1469.583333,...,1507.0,1570.0,0.027123,6205.899902,6205.899902,7.978623,0.000000,247.070007,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-06,4146.50,4153.25,4062.00,4119.50,4119.50,2310919.0,4186.000000,4156.708333,4175.3125,4153.629427,...,1793.0,1873.0,0.055488,7387.899902,7387.899902,40.930000,-0.197842,429.910004,-0.197842,0
2022-05-09,4087.25,4099.00,3970.00,3987.50,3987.50,2139992.0,4083.416667,4057.458333,4092.4375,4070.564714,...,1793.0,1873.0,0.055488,7216.600098,7216.600098,39.750000,-0.234798,417.459991,-0.234798,1
2022-05-10,3993.25,4065.50,3953.00,3996.75,3996.75,2247750.0,4034.583333,4014.125000,4022.8125,4033.657357,...,1793.0,1873.0,0.055488,7243.200195,7243.200195,39.939999,-0.230414,420.290009,-0.230414,0


In [4]:
values = df.values

In [5]:
values

array([[ 1.48525000e+03,  1.48975000e+03,  1.46225000e+03, ...,
         2.47070007e+02,  0.00000000e+00,  0.00000000e+00],
       [ 1.46700000e+03,  1.48275000e+03,  1.46675000e+03, ...,
         2.47070007e+02,  0.00000000e+00,  0.00000000e+00],
       [ 1.47875000e+03,  1.48050000e+03,  1.45025000e+03, ...,
         2.47070007e+02,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 3.99325000e+03,  4.06550000e+03,  3.95300000e+03, ...,
         4.20290009e+02, -2.30413852e-01,  0.00000000e+00],
       [ 3.99050000e+03,  4.05050000e+03,  3.92375000e+03, ...,
         4.27589996e+02, -2.45929766e-01,  0.00000000e+00],
       [ 3.93975000e+03,  3.95425000e+03,  3.89150000e+03, ...,
         4.24399994e+02, -2.48630713e-01,  0.00000000e+00]])

In [6]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    df = pd.DataFrame(data)
    df_x = df.iloc[:,:-1]
    df_y = df.iloc[:,-1:]
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df_x.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df_y.shift(-i))
    # put it all together
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [7]:
sequence_length = 5
values = series_to_supervised(values, n_in=sequence_length)

In [84]:
values

array([[ 1.48525000e+03,  1.48975000e+03,  1.46225000e+03, ...,
         2.47070007e+02,  0.00000000e+00,  0.00000000e+00],
       [ 1.46700000e+03,  1.48275000e+03,  1.46675000e+03, ...,
         2.47070007e+02,  0.00000000e+00,  0.00000000e+00],
       [ 1.47875000e+03,  1.48050000e+03,  1.45025000e+03, ...,
         2.47070007e+02,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 4.14550000e+03,  4.19575000e+03,  4.12900000e+03, ...,
         4.17459991e+02, -2.34798052e-01,  0.00000000e+00],
       [ 4.17125000e+03,  4.30300000e+03,  4.14275000e+03, ...,
         4.20290009e+02, -2.30413852e-01,  0.00000000e+00],
       [ 4.28975000e+03,  4.30075000e+03,  4.09925000e+03, ...,
         4.27589996e+02, -2.45929766e-01,  0.00000000e+00]])

## Models

In [9]:
weights = {0:1.0, 1:300.0}

lr_clf = LogisticRegression(random_state=0, max_iter=10000, class_weight=weights)

tree_clf = DecisionTreeClassifier(criterion="entropy", random_state=0, class_weight=weights)

kernel = "rbf"
c = 1
gamma = "auto"
svc_clf = SVC(kernel=kernel, C=c,gamma=gamma, probability=True, class_weight=weights)

# LightGB
number_of_positive_samples = len(df[df["target"] == 1])
total_samples = len(df)
scale_pos_weight = 100 - (number_of_positive_samples / total_samples * 100)
params = {
   "scale_pos_weight": scale_pos_weight
}
lgb_clf = LGBMClassifier(scale_pos_weight=300)

num_trees = 100
max_features = 3
rf_clf = RandomForestClassifier(n_estimators=num_trees, max_features=max_features, class_weight=weights)

num_trees = 50
seed=7
ada_clf = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)


models = []
models.append(('LR',  lr_clf))
models.append(('SVC', svc_clf))
models.append(('ABC',  ada_clf))
models.append(('DT',  tree_clf))
models.append(('RF',  rf_clf))
models.append(('XGB', lgb_clf))

In [10]:
def model_eval(model, values, n_train):
    pred = []
    y_true = []
    probs = []
    
    for i in tqdm(range(n_train, len(values))):
        
        train, test = values[0:i], values[i:i+1]
        
        # Split features, labels
        x_train = train[:,:-1]
        y_train = train[:,-1]

        x_test = test[:,:-1]
        y_test = test[:,-1]
        
        # Standardize the data
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        
        # fit data to model and predict probabilities
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        pred.append(predictions[0])
        y_true.append(y_test[0, 0])
        prob = model.predict_proba(x_test)
        probs.append(prob[:, 1])
        
        print(pred)
        print(y_true)
        

    accuracy = accuracy_score(y_true, pred)
    report = classification_report(y_true, pred, labels=[0, 1], target_names=["0", "1"], output_dict=True)
    prec_1 = report["1"]["precision"]
    rec_1 = report["1"]["recall"]
    f1_1 = report["1"]["f1-score"]        
    

    return rec_1, prec_1, f1_1, accuracy

In [11]:
results = defaultdict(dict)
n_train = 5447
n_repetitions = 20

for model in tqdm(models):
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []
    for j in tqdm(range(0, n_repetitions)):
        rec, prec, f1_score, acc = model_eval(model[1], values, n_train)
        accuracies.append(acc)
        recalls.append(rec)
        precisions.append(prec)
        f1_scores.append(f1_score)
    results[model[0]] = {'accuracy':[], 'recall':[], 'precision':[], 'f1_score':[]}
    results[model[0]]['accuracy'] = accuracies
    results[model[0]]['recall'] = recalls
    results[model[0]]['precision'] = precisions
    results[model[0]]['f1_score'] = f1_scores

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

  5%|████▏                                                                              | 1/20 [00:22<06:58, 22.00s/it]

[0.0]
[0.0]




 10%|████████▎                                                                          | 2/20 [00:44<06:42, 22.34s/it]

[0.0, 0.0]
[0.0, 0.0]




 15%|████████████▍                                                                      | 3/20 [01:04<06:01, 21.29s/it]

...

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]




 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [01:41<00:05,  5.21s/it]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]




100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:46<00:00,  5.35s/it]

100%|███████████████████████████████████████████████████████████████████████████████| 6/6 [14:55:24<00:00, 8954.02s/it]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]


In [12]:
# Running LSTM model and preparing the data

features = df[df.columns[0:-1]].to_numpy()
target = df.iloc[: , -1].to_numpy()

In [13]:
features = np.delete(features, [0], 1).astype('float32')

N = len(features)

In [14]:
features.shape

(5472, 257)

In [15]:
D = features.shape[1]

i = Input(shape=(sequence_length,D))
x = LSTM(30, return_sequences=True)(i)
x = LSTM(5, return_sequences=False)(x)
x = Dense(10)(x)
x = Dense(1, activation='sigmoid')(x)
deep_lstm = Model(i,x)

rnn_models = []
rnn_models.append(('deep_lstm', deep_lstm))

for rnn_model in rnn_models:
    accuracies = []
    recalls = []
    precisions = []
    f1_scores = []


    
    for l in range(0, n_repetitions):
        pred = []
        y_true = [] 
        probs = []
        
        # walk forward cross validation for LSTM
        
        for k in tqdm(range(n_train, len(features)-sequence_length)):
            features_train = features[:k]
            features_test = features[k:]

            scaler = StandardScaler()
            scaler.fit(features_train)
            features_train = scaler.transform(features_train)
            features_test = scaler.transform(features_test)

            # create X_train, Y_train, X_test, Y_test 
            X_train = []
            Y_train = []
            X_test = []
            Y_test = []

            for t in range(k - sequence_length):
                x = features_train[t:t+sequence_length]
                X_train.append(x)
                y = target[t+sequence_length]
                Y_train.append(y)


            X_train = np.array(X_train).reshape(-1,sequence_length,D)
            Y_train = np.array(Y_train)

            
            X_test = features_test[0:sequence_length]
            Y_test = target[k+sequence_length]


            X_test = np.array(X_test).reshape(-1,sequence_length,D)
            Y_test = np.array(Y_test)
            y_true.append(Y_test)
            rnn_model[1].compile(loss='binary_crossentropy',
                      optimizer=Adam(learning_rate=0.001),
                      metrics=[BinaryAccuracy(name='accuracy', threshold=0.5),Recall(name='recall'),Precision(name='precision')])

            class_weight = {0: 1., 1: 6.2}
            r = rnn_model[1].fit(X_train,Y_train,epochs=100,verbose=0)
            prediction = (rnn_model[1].predict(X_test)[0,0]>0.5).astype('int32')
            pred.append(prediction)
            prob = rnn_model[1].predict(X_test)[0,0]
            probs.append(prob)
            
        accuracy = accuracy_score(y_true, pred)
        report = classification_report(y_true, pred, labels=[0, 1], target_names=["0", "1"], output_dict=True)
        prec_1 = report["1"]["precision"]
        rec_1 = report["1"]["recall"]
        f1_1 = report["1"]["f1-score"]

        recalls.append(rec_1)
        precisions.append(prec_1)
        f1_scores.append(f1_1)
        accuracies.append(accuracy)

        
    results[rnn_model[0]] = {'accuracy':[], 'recall':[], 'precision':[], 'f1_score':[]}
    results[rnn_model[0]]['accuracy'] = accuracies
    results[rnn_model[0]]['recall'] = recalls
    results[rnn_model[0]]['precision'] = precisions
    results[rnn_model[0]]['f1_score'] = f1_scores

 20%|████████████████▌                                                                  | 4/20 [04:16<17:01, 63.85s/it]

 25%|████████████████████▊                                                              | 5/20 [05:20<15:53, 63.60s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [26:59<00:00, 80.98s/it]


In [16]:
# Print out the final results/metrics for each of the model

results

defaultdict(dict,
            {'LR': {'accuracy': [0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6,
               0.6],
              'recall': [0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8,
               0.8],
              'precision': [0.36363636363636365,
               0.36363636363636365,
               0.36363636363636365,
               0.36363636363636

In [20]:
# Create backup of the results because of the long execution time

results_backup = results

In [17]:
# Run Kruskal Wallis statistical test

accuracies = []
for i, model in enumerate(results.keys()):
    model_accs = results[model]['accuracy']
    accuracies.insert(i, model_accs)

result_kruskal_acc = stats.kruskal(*accuracies)


recalls = []
for i, model in enumerate(results.keys()):
    model_recs = results[model]['recall']
    recalls.insert(i, model_recs)

result_kruskal_rec = stats.kruskal(*recalls)


precisions = []
for i, model in enumerate(results.keys()):
    model_precs = results[model]['precision']
    precisions.insert(i, model_precs)

result_kruskal_prec = stats.kruskal(*precisions)


f1_scores = []
for i, model in enumerate(results.keys()):
    model_f1s = results[model]['f1_score']
    f1_scores.insert(i, model_f1s)

result_kruskal_f1_s = stats.kruskal(*f1_scores)

In [22]:
result_kruskal_acc

KruskalResult(statistic=125.01448619615115, pvalue=1.4393732766173497e-24)

In [23]:
result_kruskal_rec

KruskalResult(statistic=125.64727504064469, pvalue=1.0594503375613589e-24)

In [24]:
result_kruskal_prec

KruskalResult(statistic=117.83561825286448, pvalue=4.6400580823915384e-23)

In [25]:
result_kruskal_f1_s

KruskalResult(statistic=113.96063646239375, pvalue=3.015939170835012e-22)

In [73]:
# Run pairwise nemenyi statistical test for accuracies

models = list(results.keys())
accuracies = []
nemenyi_accuracies = pd.DataFrame()


for i in range(len(models)):
    model_accs = results[models[i]]['accuracy']
    accuracies.append(model_accs)

np_accuracies = np.array(accuracies)
nemenyi_accuracies = sp.posthoc_nemenyi_friedman(np_accuracies.T)
        

    
# Run pairwise nemenyi statistical test for recalls
        
nemenyi_recalls = defaultdict(dict)
recalls = []
nemenyi_recalls = pd.DataFrame()

for i in range(len(models)):
    model_recs = results[models[i]]['recall']
    recalls.append(model_recs)
        
np_recalls = np.array(recalls)
nemenyi_recalls = sp.posthoc_nemenyi_friedman(np_recalls.T)



# Run pairwise nemenyi statistical test for precisions
       
nemenyi_precisions = defaultdict(dict)
precisions = []
nemenyi_precisions = pd.DataFrame()

for i in range(len(models)):
    model_precs = results[models[i]]['precision']
    precisions.append(model_precs)

np_precisions = np.array(precisions)
nemenyi_precisions = sp.posthoc_nemenyi_friedman(np_precisions.T)



# Run pairwise nemenyi statistical test for f1-scores

nemenyi_f1scores = defaultdict(dict)
f1scores = []
nemenyi_f1scores = pd.DataFrame()

for i in range(len(models)):
    model_f1s = results[models[i]]['f1_score']
    f1scores.append(model_f1s)
        
np_f1scores = np.array(f1scores)
nemenyi_f1scores = sp.posthoc_nemenyi_friedman(np_f1scores.T)

In [80]:
nemenyi_accuracies.to_csv('nemenyi_accuracies.csv')

In [81]:
nemenyi_recalls.to_csv('nemenyi_recalls.csv')

In [82]:
nemenyi_precisions.to_csv('nemenyi_precisions.csv')

In [83]:
nemenyi_f1scores.to_csv('nemenyi_f1scores.csv')

In [89]:
deep_lstm_acc = pd.Series(results['deep_lstm']['accuracy'])

In [92]:
deep_lstm_acc.std()

0.07608859102526823

In [104]:
deep_lstm_acc.mean()

0.85

In [93]:
deep_lstm_rec = pd.Series(results['deep_lstm']['recall'])

In [94]:
deep_lstm_rec.std()

0.23283154063783856

In [105]:
deep_lstm_rec.mean()

0.4499999999999999

In [99]:
deep_lstm_prec = pd.Series(results['deep_lstm']['precision'])

In [100]:
deep_lstm_prec.std()

0.3834096033964306

In [106]:
deep_lstm_prec.mean()

0.775

In [102]:
deep_lstm_f1s = pd.Series(results['deep_lstm']['f1_score'])

In [103]:
deep_lstm_f1s.std()

0.2887293917392812

In [107]:
deep_lstm_f1s.mean()

0.5678571428571428